In [2]:
import cudf
import cuspatial
from cuspatial._lib.shapefile_reader import cpp_read_polygon_shapefile
result = cpp_read_polygon_shapefile("demos/small_demo.shp")
lengths = cudf.Series(result[1], name="lengths")
coords = cudf.DataFrame({"x": cudf.Series(result[2]), "y": cudf.Series(result[3])})
import numpy as np

points_x = cudf.Series([0, 1, 2, 3])
points_y = cudf.Series([0, 1, 2, 3])
# this is a bummer situation - computing prefix sum on host
r_pos = np.zeros(lengths.shape[0])
r_pos[0] = lengths[0]
for i in range(1, lengths.shape[0]):
    r_pos[i] = lengths[i] + r_pos[i-1]
r_pos = cudf.Series(r_pos).astype('int32')
f_pos = cudf.Series(np.arange(1, len(r_pos)+1)) # f_pos prefix sum of arange means no inner rings
result = cuspatial.point_in_polygon_bitmap(points_x, points_y, f_pos[0:32], r_pos[0:32], coords['x'], coords['y'])

In [3]:
result

,in_polygon_1,in_polygon_2,in_polygon_3,in_polygon_4,in_polygon_5,in_polygon_6,in_polygon_7,in_polygon_8,in_polygon_9,in_polygon_10,...,in_polygon_23,in_polygon_24,in_polygon_25,in_polygon_26,in_polygon_27,in_polygon_28,in_polygon_29,in_polygon_30,in_polygon_31,in_polygon_32
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
import geopandas
small = geopandas.read_file("demos/small_demo.shp")

breakpoint()
print(small.columns)
print(small.iloc[0].wkt)
print(small.iloc[0].geometry)
print(dir(small.iloc[0].geometry))
print(dir(small.iloc[0]))
print(small.iloc[0])
small.to_file

--Return--
> <ipython-input-20-62589f031721>(4)<module>()->None
-> breakpoint()
(Pdb) small
    name          metro        lat         lon  \
0   None  Las Vegas, NV  36.121160 -115.168623   
1   None  Las Vegas, NV  36.122534 -115.168735   
2   None  Las Vegas, NV  36.121212 -115.168340   
3   None  Las Vegas, NV  36.123085 -115.168900   
4   None  Las Vegas, NV  36.122339 -115.170400   
5   None  Las Vegas, NV  36.122100 -115.170396   
6   None  Las Vegas, NV  36.121362 -115.169265   
7   None  Las Vegas, NV  36.122369 -115.168626   
8   None  Las Vegas, NV  36.122064 -115.168503   
9   None  Las Vegas, NV  36.121889 -115.168461   
10  None  Las Vegas, NV  36.121637 -115.168466   
11  None  Las Vegas, NV  36.121282 -115.168796   
12  None  Las Vegas, NV  36.121930 -115.170149   
13  None  Las Vegas, NV  36.121837 -115.168678   
14  None  Las Vegas, NV  36.122203 -115.169901   
15  None  Las Vegas, NV  36.121469 -115.169140   
16  None  Las Vegas, NV  36.121719 -115.168547   
17  None

(Pdb) type(small)
<class 'geopandas.geodataframe.GeoDataFrame'>
(Pdb) type(small.iloc[0])
<class 'pandas.core.series.Series'>
(Pdb) type(small.iloc[0].wkt)
<class 'str'>
(Pdb) type(small.iloc[0].geometry)
<class 'shapely.geometry.polygon.Polygon'>
